# Recruitment Funnel & Time-to-Hire Optimization

Synthetic ATS dataset analysis.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

candidates = pd.read_csv('../data/candidates.csv', parse_dates=['application_date'])
stages = pd.read_csv('../data/recruitment_stages.csv', parse_dates=['stage_date'])
offers = pd.read_csv('../data/offers.csv')

candidates.head()

## Funnel counts

In [ ]:
stage_counts = stages.groupby('stage')['candidate_id'].nunique().sort_values(ascending=False)
stage_counts

In [ ]:
stage_counts.plot(kind='bar')
plt.title('Candidates per stage')
plt.xlabel('Stage')
plt.ylabel('Unique candidates')
plt.show()

## Time-to-hire (Applied → Hired)

In [ ]:
applied = stages[stages.stage=='Applied'].groupby('candidate_id', as_index=False)['stage_date'].min().rename(columns={'stage_date':'applied_date'})
hired = stages[stages.stage=='Hired'].groupby('candidate_id', as_index=False)['stage_date'].min().rename(columns={'stage_date':'hired_date'})

tth = applied.merge(hired, on='candidate_id', how='inner')
tth['time_to_hire_days'] = (tth['hired_date'] - tth['applied_date']).dt.days

tth = tth.merge(candidates[['candidate_id','role','source','location']], on='candidate_id', how='left')

tth.groupby('role')['time_to_hire_days'].agg(['count','mean','median']).sort_values('mean', ascending=False)

In [ ]:
tth['time_to_hire_days'].plot(kind='hist', bins=30)
plt.title('Distribution of time-to-hire (days)')
plt.xlabel('Days')
plt.ylabel('Hires')
plt.show()

## Bottleneck analysis (time between stages)

In [ ]:
st = stages.sort_values(['candidate_id','stage_date']).copy()
st['prev_date'] = st.groupby('candidate_id')['stage_date'].shift()
st['prev_stage'] = st.groupby('candidate_id')['stage'].shift()
st['days_between'] = (st['stage_date'] - st['prev_date']).dt.days

transitions = (st.dropna(subset=['prev_stage'])
               .groupby(['prev_stage','stage'])['days_between']
               .agg(['count','mean','median'])
               .sort_values('count', ascending=False))
transitions.head(20)

## Source effectiveness (hire rate)

In [ ]:
hired_ids = set(stages.loc[stages.stage=='Hired','candidate_id'].unique())
source_perf = candidates.groupby('source')['candidate_id'].nunique().to_frame('candidates')
source_perf['hires'] = candidates[candidates.candidate_id.isin(hired_ids)].groupby('source')['candidate_id'].nunique()
source_perf = source_perf.fillna(0)
source_perf['hire_rate_pct'] = (source_perf['hires'] / source_perf['candidates'] * 100).round(2)
source_perf.sort_values('hire_rate_pct', ascending=False)

## Offer acceptance

In [ ]:
offers2 = offers.merge(candidates[['candidate_id','role','source']], on='candidate_id', how='left')
offers2['accepted_flag'] = (offers2['accepted']=='Yes').astype(int)
offers2.groupby('role')['accepted_flag'].agg(['count','mean']).sort_values('mean', ascending=False)

In [ ]:
offers2.boxplot(column='offered_salary', by='accepted')
plt.title('Offered salary by acceptance')
plt.suptitle('')
plt.xlabel('Accepted')
plt.ylabel('Offered salary')
plt.show()

## Recommendations (write-up idea)
- Shift budget toward sources with higher hire rate (often referrals)
- Reduce bottleneck stages by adding interviewer capacity or tighter SLA
- Monitor acceptance rate by role and adjust comp bands / candidate experience
